In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split

# Load the pre-trained BERT model and tokenizer.
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# Prepare data
data = ["madam", "apple", "racecar", "hello", "level"]  # Sample data
labels = [1, 0, 1, 0, 1]  # 1 for palindrome, 0 for not

# Tokenize data
tokens = tokenizer(data, padding=True, truncation=True, return_tensors="pt")
input_ids, attention_mask = tokens['input_ids'], tokens['attention_mask']
labels = torch.tensor(labels)

# Split data into training and validation sets
train_inputs, val_inputs, train_masks, val_masks, train_labels, val_labels = train_test_split(
    input_ids, attention_mask, labels, test_size=0.1, random_state=42)

# Create TensorDatasets
train_dataset = TensorDataset(train_inputs, train_masks, train_labels)
val_dataset = TensorDataset(val_inputs, val_masks, val_labels)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [10]:
# Define an optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

# Training loop
for epoch in range(3):  # Loop over the dataset multiple times
    model.train()
    for batch in train_loader:
        inputs, masks, labels = batch
        outputs = model(input_ids=inputs, attention_mask=masks, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Validation loop
    model.eval()
    val_loss = 0
    val_accuracy = 0
    with torch.no_grad():
        for batch in val_loader:
            inputs, masks, labels = batch
            outputs = model(input_ids=inputs, attention_mask=masks, labels=labels)
            loss = outputs.loss
            logits = outputs.logits

            val_loss += loss.item()
            predictions = torch.argmax(logits, dim=1)
            val_accuracy += (predictions == labels).sum().item() / len(labels)

    print(f"Epoch {epoch+1}, Loss: {val_loss/len(val_loader)}, Accuracy: {val_accuracy/len(val_loader)}")

print("Training complete.")

Epoch 1, Loss: 0.3131941854953766, Accuracy: 1.0
Epoch 2, Loss: 0.3350169062614441, Accuracy: 1.0
Epoch 3, Loss: 0.3573446571826935, Accuracy: 1.0
Training complete.
